In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

## Logistic Regression

In [22]:
# Train logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Get model predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)
print(y_pred_proba[:5]) # Sanity check

# Calculate logg loss
logLoss = log_loss(y_test, y_pred_proba)
print(f'Log Loss: {logLoss}')

# Calculate brier score
y_pred_proba_class1 = y_pred_proba[:, 1]
brier_score = brier_score_loss(y_test, y_pred_proba_class1)
print(f'Brier Score: {brier_score}')

[[0.8324654  0.1675346 ]
 [0.46510955 0.53489045]
 [0.09471392 0.90528608]
 [0.56849522 0.43150478]
 [0.33720608 0.66279392]]
Log Loss: 0.6013579939175102
Brier Score: 0.20769966982731866


## Hyperparameter Tuning

In [23]:
# # Perform grid search
# grid = {
#     'C': [0.001, 0.1, 1, 10, 100],
#     'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
#     'max_iter': [100, 250, 500, 750, 1000],
#     'tol': [1e-4, 1e-5]
# }
# 
# gridLog = LogisticRegression()
# gridSearch = GridSearchCV(gridLog, grid, cv=5, scoring=['precision', 'recall', 'f1'], refit='f1')
# gridSearch.fit(X_train, y_train)
# print(f'Best Params: {gridSearch.best_params_}')

In [24]:
# Train with new parameters
newModel = LogisticRegression(
    C=1,
    max_iter=100,
    solver='saga',
    tol=0.0001
)
newModel.fit(X_train, y_train)

# Get model predictions
y_pred = newModel.predict(X_test)
y_pred_proba = newModel.predict_proba(X_test)

# Compute Metrics log loss and brier score
logLoss = log_loss(y_test, y_pred_proba)
print(f'Log Loss: {logLoss}')
y_pred_proba_class1 = y_pred_proba[:, 1]
brier_score = brier_score_loss(y_test, y_pred_proba_class1)
print(f'Brier Score: {brier_score}')

Log Loss: 0.6013059888683515
Brier Score: 0.20771566236729694


C:\Users\Trever\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
